In [112]:
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LassoCV
import xgboost as xgb
from scipy import sparse
import numpy as np
import pandas as pd

In [71]:
X_train_stage1 = joblib.load( 'X_train_stage1.pkl')
X_test_stage1 = joblib.load( 'X_test_stage1.pkl')
Y_train_duration = joblib.load( 'Y_train_duration.pkl')
Y_train_trajlength = joblib.load( 'Y_train_trajlength.pkl')
Y_train_price = joblib.load( 'Y_train_price.pkl')

In [72]:
n_train = X_train_stage1.shape[0]
n_test = X_test_stage1.shape[0]

In [73]:
print X_train_stage1.shape
print X_test_stage1.shape
print Y_train_duration.shape
print Y_train_trajlength.shape
print Y_train_price.shape

(465172, 1561)
(465172, 1561)
(465172,)
(465172,)
(465172,)


In [74]:
idx_train, idx_val = train_test_split(np.arange(n_train), test_size = 0.20)

In [120]:
X_test = X_test_stage1

X_full_train = X_train_stage1
Y_full_train_dur = Y_train_duration
Y_full_train_traj = Y_train_trajlength

sX_full_train = sparse.csc_matrix(X_full_train)
dtrain_full_dur = xgb.DMatrix(sX_full_train, label= Y_full_train_dur)
dtrain_full_traj = xgb.DMatrix(sX_full_train, label= Y_full_train_traj)

In [99]:
X_train, X_val = X_train_stage1[idx_train], X_train_stage1[idx_val]
Y_train_dur, Y_val_dur = Y_train_duration[idx_train], Y_train_duration[idx_val]
Y_train_traj, Y_val_traj = Y_train_trajlength[idx_train], Y_train_trajlength[idx_val]
Y_train_pri, Y_val_pri = Y_train_price[idx_train], Y_train_price[idx_val]

In [77]:
print X_train.shape
print X_val.shape
print Y_train_dur.shape
print Y_val_dur.shape
print Y_train_traj.shape
print Y_val_traj.shape

(372137, 1561)
(93035, 1561)
(372137,)
(93035,)
(372137,)
(93035,)


In [78]:
sX_train = sparse.csc_matrix(X_train)
sX_val = sparse.csc_matrix(X_val)
sX_test = sparse.csc_matrix(X_test)

In [79]:
dtest = xgb.DMatrix(sX_test)

In [80]:
dtrain_dur = xgb.DMatrix(sX_train, label= Y_train_dur)
dval_dur = xgb.DMatrix(sX_val, label=Y_val_dur)
dtrain_traj = xgb.DMatrix(sX_train, label= Y_train_traj)
dval_traj = xgb.DMatrix(sX_val, label =Y_val_traj)

In [82]:
from sklearn.metrics import make_scorer

def rmpse_loss_func(ground_truth, predictions):
    err = np.sqrt(np.mean((np.true_divide(predictions, ground_truth) - 1.)**2))
    return err

rmpse_loss  = make_scorer(rmpse_loss_func, greater_is_better=False)

In [83]:
def rmpse(preds, dtrain):
    labels = dtrain.get_label()
    err = np.sqrt(np.mean((np.true_divide(preds, labels) - 1.)**2))
    return 'error', err

In [84]:
watchlist_dur = [(dval_dur, 'eval_dur'), (dtrain_dur, 'train_dur')]
watchlist_traj = [(dval_traj, 'eval_traj'), (dtrain_traj, 'train_traj')]

# Dur

In [121]:
param = { 'objective' : "reg:linear", 
          'booster' : "gbtree",
          'eta'                 :0.01, 
          'max_depth'           :12, 
          'colsample_bytree'    : 0.7,
          'subsample' : 0.7,
          'gamma' : 1,
          'min_child_weight' : 5,
          'n_thread' : 8
        }

In [124]:
bst_dur = xgb.train(param, dtrain_full_dur, evals=[(dtrain_full_dur, 'train')], 
                num_boost_round = 350, feval= rmpse, maximize = False)

[0]	train-rmse:4.92246	train-error:0.898853
[1]	train-rmse:4.87349	train-error:0.889829
[2]	train-rmse:4.82499	train-error:0.880901
[3]	train-rmse:4.77697	train-error:0.872063
[4]	train-rmse:4.72943	train-error:0.863314
[5]	train-rmse:4.68238	train-error:0.854654
[6]	train-rmse:4.63579	train-error:0.846079
[7]	train-rmse:4.58968	train-error:0.837592
[8]	train-rmse:4.54403	train-error:0.829191
[9]	train-rmse:4.49883	train-error:0.820872
[10]	train-rmse:4.45409	train-error:0.812637
[11]	train-rmse:4.40979	train-error:0.804485
[12]	train-rmse:4.36597	train-error:0.796411
[13]	train-rmse:4.32256	train-error:0.788422
[14]	train-rmse:4.27959	train-error:0.780512
[15]	train-rmse:4.23709	train-error:0.772676
[16]	train-rmse:4.195	train-error:0.764923
[17]	train-rmse:4.15331	train-error:0.757251
[18]	train-rmse:4.11204	train-error:0.749655
[19]	train-rmse:4.07118	train-error:0.742136
[20]	train-rmse:4.03074	train-error:0.734693
[21]	train-rmse:3.99072	train-error:0.727321
[22]	train-rmse:3.9510

[182]	train-rmse:0.847359	train-error:0.14907
[183]	train-rmse:0.839994	train-error:0.147716
[184]	train-rmse:0.832709	train-error:0.146378
[185]	train-rmse:0.825495	train-error:0.145053
[186]	train-rmse:0.81837	train-error:0.143744
[187]	train-rmse:0.811299	train-error:0.142449
[188]	train-rmse:0.804313	train-error:0.141169
[189]	train-rmse:0.797405	train-error:0.139903
[190]	train-rmse:0.790577	train-error:0.138653
[191]	train-rmse:0.783843	train-error:0.137417
[192]	train-rmse:0.777167	train-error:0.136194
[193]	train-rmse:0.770561	train-error:0.134984
[194]	train-rmse:0.764071	train-error:0.133792
[195]	train-rmse:0.757618	train-error:0.132611
[196]	train-rmse:0.75124	train-error:0.131443
[197]	train-rmse:0.744951	train-error:0.13029
[198]	train-rmse:0.738742	train-error:0.129152
[199]	train-rmse:0.732584	train-error:0.128025
[200]	train-rmse:0.726529	train-error:0.126914
[201]	train-rmse:0.720514	train-error:0.125813
[202]	train-rmse:0.714594	train-error:0.124727
[203]	train-rmse:

In [126]:
rf_dur = RandomForestRegressor(max_depth = 22, max_features = 'sqrt', n_estimators=1000, verbose = 3, n_jobs = -1, criterion='mse').fit(sX_full_train, Y_full_train_dur)

building tree 1 of 1000
 building tree 2 of 1000
building tree 3 of 1000 building tree 4 of 1000

building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   44.5s


building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000
building tree 46 of 1000
building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000
building tree 52 of 1000
building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000
building tree 60 of 1000
building tree 61 of 1000
building tree 62 of 1000
building tree 63 of 1000


[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  4.0min


building tree 121 of 1000
building tree 122 of 1000
building tree 123 of 1000
building tree 124 of 1000
building tree 125 of 1000
building tree 126 of 1000
building tree 127 of 1000
building tree 128 of 1000
building tree 129 of 1000
building tree 130 of 1000
building tree 131 of 1000
building tree 132 of 1000
building tree 133 of 1000
building tree 134 of 1000
building tree 135 of 1000
building tree 136 of 1000
building tree 137 of 1000
building tree 138 of 1000
building tree 139 of 1000
building tree 140 of 1000
building tree 141 of 1000
building tree 142 of 1000
building tree 143 of 1000
building tree 144 of 1000
building tree 145 of 1000
building tree 146 of 1000
building tree 147 of 1000
building tree 148 of 1000
building tree 149 of 1000
building tree 150 of 1000
building tree 151 of 1000
building tree 152 of 1000
building tree 153 of 1000
building tree 154 of 1000
building tree 155 of 1000
building tree 156 of 1000
building tree 157 of 1000
building tree 158 of 1000
building tre

...

[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed: 18.2min


building tree 504 of 1000
building tree 505 of 1000
building tree 506 of 1000
building tree 507 of 1000
building tree 508 of 1000
building tree 509 of 1000
building tree 510 of 1000
building tree 511 of 1000
building tree 512 of 1000
building tree 513 of 1000
building tree 514 of 1000
building tree 515 of 1000
building tree 516 of 1000
building tree 517 of 1000
building tree 518 of 1000
building tree 519 of 1000
building tree 520 of 1000
building tree 521 of 1000
building tree 522 of 1000
building tree 523 of 1000
building tree 524 of 1000
building tree 525 of 1000
building tree 526 of 1000
building tree 527 of 1000
building tree 528 of 1000
building tree 529 of 1000
building tree 530 of 1000
building tree 531 of 1000
building tree 532 of 1000
building tree 533 of 1000
building tree 534 of 1000
building tree 535 of 1000
building tree 536 of 1000
building tree 537 of 1000
building tree 538 of 1000
building tree 539 of 1000
building tree 540 of 1000
building tree 541 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 28.5min


building tree 792 of 1000
building tree 793 of 1000
building tree 794 of 1000
building tree 795 of 1000
building tree 796 of 1000
building tree 797 of 1000
building tree 798 of 1000
building tree 799 of 1000
building tree 800 of 1000
building tree 801 of 1000
building tree 802 of 1000
building tree 803 of 1000
building tree 804 of 1000
building tree 805 of 1000
building tree 806 of 1000
building tree 807 of 1000
building tree 808 of 1000
building tree 809 of 1000
building tree 810 of 1000
building tree 811 of 1000
building tree 812 of 1000
building tree 813 of 1000
building tree 814 of 1000
building tree 815 of 1000
building tree 816 of 1000
building tree 817 of 1000
building tree 818 of 1000
building tree 819 of 1000
building tree 820 of 1000
building tree 821 of 1000
building tree 822 of 1000
building tree 823 of 1000
building tree 824 of 1000
building tree 825 of 1000
building tree 826 of 1000
building tree 827 of 1000
building tree 828 of 1000
building tree 829 of 1000
building tre

KeyboardInterrupt: 

In [ ]:
X_ensemble_dur = np.zeros((n_train, 2))
X_ensemble_dur[:,0] = bst_dur.predict(dtrain_full_dur)
X_ensemble_dur[:,1] = rf_dur.predict(sX_full_train)
lasso_dur = LassoCV().fit(X_ensemble_dur, Y_full_train_dur)

In [ ]:
joblib.dump(bst_dur, 'bst_dur_stage2.pkl')
joblib.dump(rf_dur, 'rf_dur_stage2.pkl')
joblib.dump(lasso_dur, 'lasso_dur_stage2.pkl')

In [128]:
#bst_train = rmpse_loss_func(Y_train_dur ,bst_dur.predict(dtrain_dur))
#bst_val = rmpse_loss_func(Y_val_dur,bst_dur.predict(dval_dur))
#print -bst_train
#print -bst_val

In [129]:
# print rmpse_loss(rf_dur, sX_train, Y_train_dur)
# print rmpse_loss(rf_dur, sX_val, Y_val_dur)

In [130]:
# X_train_ens = np.zeros((372137, 2))
# X_train_ens[:,0] = bst_dur.predict(dtrain_dur)
# X_train_ens[:,1] = rf_dur.predict(sX_train)
# X_val_ens = np.zeros((93035, 2))
# X_val_ens[:,0] = bst_dur.predict(dval_dur)
# X_val_ens[:,1] = rf_dur.predict(sX_val)
# print rmpse_loss(lasso_dur, X_train_ens, Y_train_dur)
# print rmpse_loss(lasso_dur, X_val_ens, Y_val_dur)

# TRAJ

In [ ]:
param = { 'objective' : "reg:linear", 
          'booster' : "gbtree",
          'eta'                 :0.02, 
          'max_depth'           :20, 
          'colsample_bytree'    : 0.7,
          'subsample' : 0.7,
          'gamma' : 1,
          'min_child_weight' : 5,
          'n_thread' : 8
        }

In [ ]:
bst_traj = xgb.train(param, dtrain_full_traj, evals=[(dtrain_full_traj, 'train')], 
                num_boost_round = 350, feval= rmpse, maximize = False)

In [ ]:
rf_traj = RandomForestRegressor(max_depth = 22, max_features = 'sqrt', n_estimators=1000, verbose = 3, n_jobs = -1, criterion='mse').fit(sX_full_train, Y_full_train_traj)

In [ ]:
X_ensemble_traj = np.zeros((n_train, 2))
X_ensemble_traj[:,0] = bst_traj.predict(dtrain_full_traj)
X_ensemble_traj[:,1] = rf_traj.predict(sX_full_train)
lasso_traj = LassoCV().fit(X_ensemble_traj, Y_full_train_traj)

In [ ]:
joblib.dump(bst_traj, 'bst_traj_stage2.pkl')
joblib.dump(rf_traj, 'rf_traj_stage2.pkl')
joblib.dump(lasso_traj, 'lasso_traj_stage2.pkl')

In [131]:
# bst_train = rmpse_loss_func(Y_train_traj ,bst_traj.predict(dtrain_traj))
# bst_val = rmpse_loss_func(Y_val_traj,bst_traj.predict(dval_traj))
# print -bst_train
# print -bst_val

In [132]:
# print rmpse_loss(rf_traj, sX_train, Y_train_traj)
# print rmpse_loss(rf_traj, sX_val, Y_val_traj)

In [133]:
# X_train_ens = np.zeros((372137, 2))
# X_train_ens[:,0] = bst_traj.predict(dtrain_traj)
# X_train_ens[:,1] = rf_traj.predict(sX_train)
# X_val_ens = np.zeros((93035, 2))
# X_val_ens[:,0] = bst_traj.predict(dval_traj)
# X_val_ens[:,1] = rf_traj.predict(sX_val)
# print rmpse_loss(lasso_traj, X_train_ens, Y_train_traj)
# print rmpse_loss(lasso_traj, X_val_ens, Y_val_traj)

# Validation


In [85]:
# X_val_ens = np.zeros((93035, 2))
# X_val_ens[:,0] = bst_dur.predict(dval_dur)
# X_val_ens[:,1] = rf_dur.predict(sX_val)
# Y_val_dur_pred = lasso_dur.predict(X_val_ens)

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    2.0s finished


In [86]:
# X_val_ens = np.zeros((93035, 2))
# X_val_ens[:,0] = bst_traj.predict(dval_traj)
# X_val_ens[:,1] = rf_traj.predict(sX_val)
# Y_val_traj_pred = lasso_traj.predict(X_val_ens)

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.7s finished


In [101]:
# Y_val_pri_pred = np.exp(Y_val_dur_pred) + np.exp(Y_val_traj_pred)

In [105]:
# Y_val_pri_pred

array([294.66929039, 348.21387242, 479.11980346, ..., 367.7081731 ,
       303.75860862, 360.37284109])

In [106]:
# np.exp(Y_val_pri)

array([372., 336., 515., ..., 435., 262., 215.])

In [107]:
# rmpse_loss_func(np.exp(Y_val_pri), Y_val_pri_pred)

0.20196613102493846

# Combine

In [108]:
X_test_ens = np.zeros((n_test, 2))
X_test_ens[:,0] = bst_dur.predict(dtest)
X_test_ens[:,1] = rf_dur.predict(sX_test)
Y_test_dur_pred = lasso_dur.predict(X_test_ens)

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    2.8s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    6.6s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   12.2s finished


In [109]:
X_test_ens = np.zeros((n_test, 2))
X_test_ens[:,0] = bst_traj.predict(dtest)
X_test_ens[:,1] = rf_traj.predict(sX_test)
Y_test_traj_pred = lasso_traj.predict(X_test_ens)

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    4.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    7.8s finished


In [110]:
Y_test_pri_pred = np.exp(Y_test_dur_pred) + np.exp(Y_test_traj_pred)

In [116]:
test_id = pd.read_csv("test.csv").ID.values

In [117]:
test_id

array([465173, 465174, 465175, ..., 930342, 930343, 930344])

In [119]:
data = {'ID': test_id,
       'PRICE': Y_test_pri_pred}
submission_df = pd.DataFrame(data = data)
submission_df.to_csv("stage_2_v0.csv", index=False)

In [127]:
joblib.dump(Y_test_dur_pred, 'Y_dur_stage2')
joblib.dump(Y_test_traj_pred, 'Y_traj_stage2')

['Y_traj_stage2']